<img src='https://github.com/jtobelem-simplon/prepa-dp100/blob/master/images/top.png?raw=true'>

# Configuration (à lancer avant tous les notebooks)

In [ ]:
# version de python
import platform
platform.python_version()

In [ ]:
# la liste des packages installés
!conda list

In [ ]:
# version de la SDK azureml
import azureml.core
print("Ready to use Azure ML", azureml.core.VERSION)

Si le notebook est executé en dehors d'Azure, il faut télécharger le fichier config.json depuis le portail https://portal.azure.com/, et le mettre dans le workspace qui contient le notebook.

Si le notebook est exécuté directement depuis le workspace Azure, le fichier de config devrait déjà être là.

In [ ]:
# connexion au workspace
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, "loaded")

# Création du script

In [ ]:
import os, shutil

# Create a folder for the experiment files
training_folder = 'titanic-training'
os.makedirs(training_folder, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('data/titanic/train.csv', os.path.join(training_folder, "titanic.csv"))

In [ ]:
%%writefile $training_folder/titanic_training.py
from azureml.core import Run
import joblib

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score

# Get the experiment run context
run = Run.get_context()

train_data = pd.read_csv("titanic.csv")

features = ["Age","Pclass","SibSp", "Parch", "Fare","Sex", "Embarked"]

X = pd.get_dummies(train_data[features])
y = train_data["Survived"]

# missing values
imputer = SimpleImputer(strategy='most_frequent')
imputed_X = pd.DataFrame(imputer.fit_transform(X))
imputed_X.columns = X.columns
imputed_X[["Age","Pclass","SibSp", "Parch", "Fare"]] = imputed_X[["Age","Pclass","SibSp", "Parch", "Fare"]].astype('int')


# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(imputed_X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# model
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X_train, y_train)


# score
predictions = model.predict(X_valid)

mae = mean_absolute_error(predictions.astype('int'), y_valid)
acc = accuracy_score(y_valid, predictions.astype('int'))
print("mae : {}, accuracy : {}".format(mae, acc))
run.log('mae', mae)
run.log('acc',acc)

# Save the trained model in the outputs folder
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/titanic_model.pkl')

run.complete()

# Entrainement d'un modèle sur la machine locale

[tutoriel microsoft : partie 1](https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-train-models-with-aml)

[tutoriel microsoft : partie 2](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-train-ml-models)

- [experiment dans la SDK](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.experiment.experiment?view=azure-ml-py)

- [estimator dans la SDK](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.estimator.estimator?view=azure-ml-py)

NB : en cas d'erreur avec docker, https://askubuntu.com/questions/477551/how-can-i-use-docker-without-sudo

In [ ]:
from azureml.train.estimator import Estimator
from azureml.core import Experiment

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace = ws, name = "titanic-training-experiment")

# Create an estimator
estimator = Estimator(source_directory=training_folder,
                      entry_script='titanic_training.py',
                      compute_target='local',
                      conda_packages=['scikit-learn']
                      )

# Run the experiment based on the estimator
run = experiment.submit(config=estimator)
run.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

# Entrainement d'un modèle sur un cluster distant


Provisionnement ou récupération d'un cluster existant appelé "aml-cluster". Voir [tutoriel microsoft : partie 1](https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-train-models-with-aml#train-on-a-remote-cluster)

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "aml-cluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                                min_nodes=compute_min_nodes,
                                                                max_nodes=compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(
        ws, compute_name, provisioning_config)

    # can poll for a minimum number of nodes and for a specific timeout.
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(
        show_output=True, min_node_count=None, timeout_in_minutes=20)

    # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

Même estimator que dans la partie précédente mais avec le cluster dans la compute_target. Il va falloir provisionner le cluster (normalement 0 noeuds actifs initialement), cela peut prendre un peu de temps...

In [ ]:
from azureml.train.estimator import Estimator
from azureml.core import Experiment

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace = ws, name = "titanic-training-experiment")

# Create an estimator
estimator = Estimator(source_directory=training_folder,
                      entry_script='titanic_training.py',
                      compute_target=compute_target,
                      conda_packages=['scikit-learn']
                      )

# Run the experiment based on the estimator
run = experiment.submit(config=estimator)
run.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

# Enregistrer le modèle

In [ ]:
from azureml.core import Model

# Register the model
run.register_model(model_path='outputs/titanic_model.pkl', model_name='titanic_model',
                   tags={'Training context':'Estimator'},
                   properties={'Mean Absolute Error': run.get_metrics()['mae'], 'Accuracy': run.get_metrics()['acc']})

In [ ]:
# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

# Ne pas oublier à la fin de l'expérience!!
(si votre travail à utilisé une instance de calcul)

<img src='https://github.com/jtobelem-simplon/prepa-dp100/blob/master/images/down.png?raw=true'>



In [ ]:
# stop toutes les instances de calcul
from azureml.core.compute import ComputeTarget, AmlCompute, ComputeInstance
from azureml.core.compute_target import ComputeTargetException

for compute in ComputeTarget.list(ws):
    if type(compute) is ComputeInstance and compute.get_status().state == 'Running':
        print('try to stop compute', compute.name)
        compute.stop(show_output=True)

In [ ]:
# liste tous les compute pour vérifier qu'elles sont éteintes
for compute in ComputeTarget.list(ws):
    if type(compute) is ComputeInstance:
        print(compute.name, compute.get_status())

# Ressources

[api azure](https://docs.microsoft.com/en-us/python/api/azureml-core)

[parcours d'apprentissage microsoft](https://docs.microsoft.com/fr-fr/learn/paths/build-ai-solutions-with-azure-ml-service/)

[le repository microsoft](https://github.com/MicrosoftDocs/mslearn-aml-labs.git)